In [1]:
# setup
from IPython.core.display import display,HTML
display(HTML('<style>.prompt{width: 0px; min-width: 0px; visibility: collapse}</style>'))
display(HTML(open('../rise.css').read()))

# imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(style="whitegrid", font_scale=1.5, rc={'figure.figsize':(12, 6)})


# CMPS 2200
# Introduction to Algorithms

## All pairs shortest path



## All pairs shortest path  (APSP)

Up to now, we've consider the shortest paths from a fixed source node $s$.

Of course, we will often want to know the shortest path from **all** nodes in the graph.

How can we use algorithms we have seen to do this?

Just run Bellman-Ford $|V|$ times, once per vertex.

Since one run of Bellman-Ford has $O(|V| \cdot |E|)$ work, the total work is $O(|V|^2 \cdot |E|)$

In [12]:
from heapq import heappush, heappop 
import math

def dijkstra(graph, source):
    def dijkstra_helper(visited, frontier):
        if len(frontier) == 0:
            return visited
        else:
            # Pick next closest node from heap
            distance, node = heappop(frontier)
            if node in visited:
                # Already visited, so ignore this longer path
                return dijkstra_helper(visited, frontier)
            else:
                # We now know the shortest path from source to node.
                # insert into visited dict.
                visited[node] = distance
                # Visit each neighbor of node and insert into heap.
                # We may add same node more than once, heap
                # will keep shortest distance prioritized.
                for neighbor, weight in graph[node]:
                    heappush(frontier, (distance + weight, neighbor))                
                return dijkstra_helper(visited, frontier)
        
    frontier = []
    heappush(frontier, (0, source))
    visited = dict()  # store the final shortest paths for each node.
    return dijkstra_helper(visited, frontier)



def bellmanford(graph, source):
    def bellmanford_helper(distances, k):        
        if k == len(graph): # negative cycle
            return -math.inf
        else:
            # compute new distances
            new_distances = compute_distances(graph, distances)
            
            # check if distances have converged
            if converged(distances, new_distances):
                return distances
            else:                
                return bellmanford_helper(new_distances, k+1)
        
    # initialize
    distances = dict()
    for v in graph:
        if v == source:
            distances[v] = 0
        else:
            distances[v] = math.inf
    return bellmanford_helper(distances, 0)

def compute_distances(graph, distances):
    new_distances = {}
    for v, in_neighbors in graph.items(): # this loop can be done in parallel
        # compute all possible distances from s->v
        v_distances = [distances[v]] 
        for in_neighbor, weight in in_neighbors:
            v_distances.append(distances[in_neighbor] + weight)
        new_distances[v] = min(v_distances)
    return new_distances

def converged(old_distances, new_distances):
    for k in old_distances:
        if old_distances[k] != new_distances[k]:
            return False
    return True


### Converting a graph with negative edge weights to one with only positive weights

<br>

<center>
<img src="figures/johnson1.jpg"/>
</center>    

<center>
<img src="figures/johnson2.jpg"/>
</center>    
    


In [25]:
from collections import defaultdict
from pprint import pprint

def johnson(graph):
    # add "dummy" node that links to all nodes with 0 cost.
    for v, in_neighbors in graph.items():
        in_neighbors.add(('s', 0))
    graph['s'] = {}
    
    # call bellmanford with new graph
    distances = bellmanford(graph, 's')
    print('bellman ford distances')
    print(distances)
    
    # create new graph with adjusted weights.
    # b/c we'll be using Dijkstra, we make a dict from node to out-neighbors.
    new_graph = defaultdict(set)
    for v, in_neighbors in graph.items():
        for in_neighbor, weight in in_neighbors:
            new_graph[in_neighbor].add((v, weight + distances[in_neighbor] - distances[v]))
    print('\nnew graph')
    pprint(dict(new_graph))
    
    # now, run dijkstra on each node in the graph
    print('\nunadjusted shortest paths')
    apsp = {}
    for v in new_graph:
        if v != 's':
            distances_from_v = dijkstra(new_graph, v)
            # update final path scores
            print(v, distances_from_v)            
            for u, dist in distances_from_v.items():
                distances_from_v[u] = dist - distances[v] + distances[u]
            apsp[v] = distances_from_v
    print('\nadjusted shortest paths')
    pprint(result)
    return apsp

graph = {
            'a': {('c', 2)},
            'b': {('a', -3)},
            'c': {('a', 1), ('b', 3), ('d', 3)}, 
            'd': {('b', -1)},
        }
    
result = johnson(graph)

bellman ford distances
{'a': 0, 'b': -3, 'c': -1, 'd': -4, 's': 0}

new graph
{'a': {('b', 0), ('c', 2)},
 'b': {('d', 0), ('c', 1)},
 'c': {('a', 1)},
 'd': {('c', 0)},
 's': {('d', 4), ('a', 0), ('c', 1), ('b', 3)}}

unadjusted shortest paths
c {'c': 0, 'a': 1, 'b': 1, 'd': 1}
a {'a': 0, 'b': 0, 'd': 0, 'c': 0}
b {'b': 0, 'd': 0, 'c': 0, 'a': 1}
d {'d': 0, 'c': 0, 'a': 1, 'b': 1}

adjusted shortest paths
{'a': {'a': 0, 'b': -3, 'c': -1, 'd': -4},
 'b': {'a': 4, 'b': 0, 'c': 2, 'd': -1},
 'c': {'a': 2, 'b': -1, 'c': 0, 'd': -2},
 'd': {'a': 5, 'b': 2, 'c': 3, 'd': 0}}


- motivate
- just running bellman-ford n times
- path potentials
- johnson's algorithm